In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

params = {
    'k1': 77.27,
    'k2': 8.375 * 1e-6,
    'k4': 0.161,
}


def f(t, y):
    k1 = params['k1']
    k2 = params['k2']
    k3 = 1 / params['k1']
    k4 = params['k4']

    y1, y2, y3 = y

    dy1 = k1 * (y2 - y1 * y2 + y1 - k2 * y1 ** 2)
    dy2 = k3 * (-y2 - y1 * y2 + y3)
    dy3 = k4 * (y1 - y3)

    return np.array([dy1, dy2, dy3])


def euler(f, y0, a, b, n=100):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.zeros((n + 1, len(y0)))
    y[0] = y0

    for i in range(n):
        y[i + 1] = y[i] + h * np.array(f(x, y[i]))

    return x, y


def euler_recalc(f, y0, a, b, n=100):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.zeros((n + 1, len(y0)))
    y[0] = y0

    for i in range(n):
        predict = y[i] + h * np.array(f(x[i], y[i]))
        y[i + 1] = y[i] + (np.array(f(x[i], y[i])) + np.array(f(x[i + 1], predict))) * h / 2

    return x, y


def rk4(f, y0, a, b, n=100):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.zeros((n + 1, len(y0)))
    y[0] = y0

    for i in range(n):
        phi_1 = h * np.array(f(x[i], y[i]))
        phi_2 = h * np.array(f(x[i] + h / 2, y[i] + phi_1 / 2))
        phi_3 = h * np.array(f(x[i] + h / 2, y[i] + phi_2 / 2))
        phi_4 = h * np.array(f(x[i] + h, y[i] + phi_3))

        y[i + 1] = y[i] + (phi_1 + 2 * phi_2 + 2 * phi_3 + phi_4) / 6

    return x, y


def ab(f, y0, a, b, n=100):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.zeros((n + 1, len(y0)))
    y[0] = y0
    f_list = np.zeros((n + 1, len(y0)))

    for i in range(n):
        f_list[i] = np.array(f(x[i], y[i]))

        if i < 3:
            phi_1 = h * np.array(f(x[i], y[i]))
            phi_2 = h * np.array(f(x[i] + h / 2, y[i] + phi_1 / 2))
            phi_3 = h * np.array(f(x[i] + h / 2, y[i] + phi_2 / 2))
            phi_4 = h * np.array(f(x[i] + h, y[i] + phi_3))

            y[i + 1] = y[i] + (phi_1 + 2 * phi_2 + 2 * phi_3 + phi_4) / 6

        else:
            y[i + 1] = y[i] + h * (55 * f_list[i] - 59 * f_list[i - 1] + 37 * f_list[i - 2] - 9 * f_list[i - 3]) / 24

    return x, y


def abm(f, y0, a, b, n=100):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = np.zeros((n + 1, len(y0)))
    y[0] = y0
    f_list = np.zeros((n + 1, len(y0)))

    for i in range(n):
        f_list[i] = np.array(f(x[i], y[i]))

        if i < 3:
            phi_1 = h * np.array(f(x[i], y[i]))
            phi_2 = h * np.array(f(x[i] + h / 2, y[i] + phi_1 / 2))
            phi_3 = h * np.array(f(x[i] + h / 2, y[i] + phi_2 / 2))
            phi_4 = h * np.array(f(x[i] + h, y[i] + phi_3))

            y[i + 1] = y[i] + (phi_1 + 2 * phi_2 + 2 * phi_3 + phi_4) / 6

        else:
            predict = y[i] + h * (55 * f_list[i] - 59 * f_list[i - 1] + 37 * f_list[i - 2] - 9 * f_list[i - 3]) / 24
            y[i + 1] = y[i] + h * (
                    9 * np.array(f(x[i + 1], predict)) + 19 * f_list[i] - 5 * f_list[i - 1] + f_list[i - 2]) / 24

    return x, y

In [3]:
# import numpy as np
# import csv
# 
# # Функция вычисления Якобиана через конечные разности
# def jacobian(f, t, y, eps=1e-8):
#     m = len(y)
#     J = np.zeros((m, m))
#     f0 = f(t, y)
#     for j in range(m):
#         y_eps = y.copy()
#         y_eps[j] += eps
#         df = f(t, y_eps) - f0
#         J[:, j] = df / eps
#     return J
# 
# # Неявный метод Рунге–Кутта 2-го порядка (Gauss–Legendre), используется в предикторе
# def irk2(f, y0, a, b, n=1, tol=1e-10, max_iter=10):
#     h = (b - a) / n
#     t = np.linspace(a, b, n+1)
#     m = len(y0)
#     y = np.zeros((n+1, m))
#     y[0] = y0
# 
#     for i in range(n):
#         ti, yi = t[i], y[i]
#         ti1    = t[i+1]
#         # стартовое приближение (явный Эйлер)
#         Y = yi + h * f(ti, yi)
# 
#         # ньютонианские итерации для IRK2
#         for _ in range(max_iter):
#             F_iso = f(ti1, Y)
#             G      = Y - yi - (h/2)*(f(ti, yi) + F_iso)
#             Jf     = jacobian(f, ti1, Y)
#             Jg     = np.eye(m) - (h/2)*Jf
#             # регуляризация
#             Jg    += 1e-8 * np.eye(m)
#             delta  = np.linalg.solve(Jg, G)
#             Y     -= delta
#             if np.linalg.norm(delta) < tol:
#                 break
# 
#         y[i+1] = Y
# 
#     return t, y
# 
# # Чтение первых трёх точек y[1..3] из большого CSV-файла
# def read_initial_from_csv(filename, a, b, n):
#     # Параметры большого решения: от 0 до 10 за 32e6 шагов
#     T_big = 10.0
#     N_big = 32_000_000
#     H_big = T_big / N_big
# 
#     h = (b - a) / n
#     idxs = []
#     for j in (1,2,3):
#         tj = a + j*h
#         idx = int(round(tj / H_big))
#         idxs.append(idx)
# 
#     y_init = []
#     with open(filename, 'r') as f:
#         reader = csv.reader(f)
#         for i, row in enumerate(reader):
#             if i in idxs:
#                 y_init.append([float(val) for val in row])
#                 if len(y_init) == 3:
#                     break
# 
#     return np.array(y_init)  # shape (3, dim)
# 
# # Сам метод BDF4 c комбинированным предиктором и регуляризацией
# def bdf4(f, y0, a, b, n=100, tol=1e-8, max_iter=10,
#          init_csv="Adams_Bashfort_Molton_N_32000001_k_3_T_10.csv"):
#     """
#     Метод Гира 4-го порядка (BDF4):
#       (25 y_{n+1} - 48 y_n +36 y_{n-1} -16 y_{n-2} +3 y_{n-3})/(12h)
#         = f(t_{n+1}, y_{n+1})
#     Инициализация y[1..3] берутся из CSV, предиктор — усреднённый AB4+IRK2.
#     """
#     h = (b - a) / n
#     t = np.linspace(a, b, n+1)
#     m = len(y0)
#     y = np.zeros((n+1, m))
#     y[0] = y0
# 
#     # 1) Инициализация из CSV
#     y[1:4] = read_initial_from_csv(init_csv, a, b, n)
# 
#     # коэффициенты BDF4
#     alpha = [-48/25, 36/25, -16/25, 3/25]
#     beta  = 12/25
# 
#     # 2) Основной цикл
#     for i in range(3, n):
#         ti1  = t[i+1]
#         y_nm1 = y[i]    # y_n
#         y_nm2 = y[i-1]  # y_{n-1}
#         y_nm3 = y[i-2]  # y_{n-2}
#         y_nm4 = y[i-3]  # y_{n-3}
# 
#         # AB4-предиктор
#         Y_ab = y_nm1 + h*(55*f(t[i],   y_nm1)
#                          -59*f(t[i-1], y_nm2)
#                          +37*f(t[i-2], y_nm3)
#                          - 9*f(t[i-3], y_nm4))/24
# 
#         # IRK2-предиктор
#         _, y_irk = irk2(f, y_nm1, t[i], ti1, n=1, tol=tol, max_iter=max_iter)
#         Y_irk = y_irk[-1]
# 
#         # комбинированный предиктор
#         Y = 0.5*(Y_ab + Y_irk)
# 
#         # Ньютонианская корректировка
#         for _ in range(max_iter):
#             F = f(ti1, Y)
#             G = (Y
#                  + alpha[0]*y_nm1
#                  + alpha[1]*y_nm2
#                  + alpha[2]*y_nm3
#                  + alpha[3]*y_nm4
#                 ) - beta*h*F
# 
#             Jf = jacobian(f, ti1, Y)
#             Jg = np.eye(m) - beta*h * Jf
#             # регуляризация
#             Jg += 1e-8 * np.eye(m)
# 
#             delta = np.linalg.solve(Jg, G)
#             Y    -= delta
#             if np.linalg.norm(delta) < tol:
#                 break
# 
#         y[i+1] = Y
# 
#     return t, y

In [4]:
import numpy as np


def jacobian(f, t, y, eps=1e-8):
    """
    Численный Якобиан ∂f/∂y в точке (t,y).
    """
    m = len(y)
    J = np.zeros((m, m))
    f0 = f(t, y)
    for j in range(m):
        y_eps = y.copy()
        y_eps[j] += eps
        df = f(t, y_eps) - f0
        J[:, j] = df / eps
    return J


def rk4_step(f, t, y, h):
    """
    Один шаг явного Рунге–Кутты 4-го порядка от (t,y) с шагом h.
    """
    k1 = f(t, y)
    k2 = f(t + h / 2, y + h * k1 / 2)
    k3 = f(t + h / 2, y + h * k2 / 2)
    k4 = f(t + h, y + h * k3)
    return y + h * (k1 + 2 * k2 + 2 * k3 + k4) / 6


def bdf4(f, y0, a, b, n=100, tol=1e-8, max_iter=10):
    """
    GEAR 4-го порядка (BDF4) со стартером RK4 и предиктором RK4.
    """
    h = (b - a) / n
    t = np.linspace(a, b, n + 1)
    m = len(y0)
    y = np.zeros((n + 1, m))
    y[0] = y0

    # 1) Инициализация первых трёх точек методом RK4
    for j in range(3):
        y[j + 1] = rk4_step(f, t[j], y[j], h)

    # коэффициенты BDF4:
    # (25 y_{n+1} - 48 y_n + 36 y_{n-1} - 16 y_{n-2} + 3 y_{n-3})/(12h) = f(t_{n+1},y_{n+1})
    alpha = [-48 / 25, 36 / 25, -16 / 25, 3 / 25]
    beta = 12 / 25

    # 2) Основной цикл BDF4
    for i in range(3, n):
        ti1 = t[i + 1]
        y_nm1 = y[i]  # y_n
        y_nm2 = y[i - 1]  # y_{n-1}
        y_nm3 = y[i - 2]  # y_{n-2}
        y_nm4 = y[i - 3]  # y_{n-3}

        # предиктор: один шаг RK4 от (t_i, y_i)
        Y = rk4_step(f, t[i], y_nm1, h)

        # Ньютонианский корректор
        for _ in range(max_iter):
            F = f(ti1, Y)
            G = (Y
                 + alpha[0] * y_nm1
                 + alpha[1] * y_nm2
                 + alpha[2] * y_nm3
                 + alpha[3] * y_nm4
                 ) - beta * h * F

            Jf = jacobian(f, ti1, Y)
            Jg = np.eye(m) - beta * h * Jf
            # лёгкая регуляризация, чтобы избежать вырожденности
            Jg += 1e-8 * np.eye(m)

            delta = np.linalg.solve(Jg, G)
            Y -= delta
            if np.linalg.norm(delta) < tol:
                break

        y[i + 1] = Y

    return t, y

In [5]:
from scipy.integrate import solve_ivp

# пример использования:
# y123 = read_initial_from_csv(
#     "Adams_Bashfort_Molton_N_32000001_k_3_T_10.csv",
#     a=0.0, b=10.0, n=100000
# )
# 
# print(y123)

y0 = [4.0, 1.1, 4.0]
a, b = 0.0, 300.0

In [6]:
from time import time

METHOD_ORDER = {
    'euler': 1,
    'euler_recalc': 2,
    'rk4': 4,
    'ab': 4,
    'abm': 4,
    'irk2': 2,
    'bdf2': 2,
    'bdf4': 4,
}


def solve_step(f, y0, a, b, n, method):
    if method == 'euler':
        return euler(f, y0, a, b, n)
    elif method == 'euler_recalc':
        return euler_recalc(f, y0, a, b, n)
    elif method == 'rk4':
        return rk4(f, y0, a, b, n)
    elif method == 'ab':
        return ab(f, y0, a, b, n)
    elif method == 'abm':
        return abm(f, y0, a, b, n)
    # elif method == 'irk2':
    #     return irk2(f, y0, a, b, n)
    # elif method == 'bdf2':
    #     return bdf2(f, y0, a, b, n)
    elif method == 'bdf4':
        return bdf4(f, y0, a, b, n)
    else:
        raise ValueError(f'Неизвестный метод: {method}')


def table_runge(f, y0, a, b, method, n_start=100, k_tables=6):
    n_list = [n_start * 2 ** i for i in range(k_tables)]
    order = METHOD_ORDER[method]

    rows = []

    x1, y1 = solve_step(f, y0, a, b, n_start, method)
    x2, y2 = solve_step(f, y0, a, b, 2 * n_start, method)

    diff = y2[::2] - y1
    err_runge = np.max(np.linalg.norm(diff, ord=np.inf, axis=1)) * (2 ** order) / (2 ** order - 1)

    h = (b - a) / n_start
    rows.append([n_start, h, err_runge, None])
    prev_err_r = err_runge

    for n in n_list[1:]:
        y1 = y2.copy()
        x2, y2 = solve_step(f, y0, a, b, 2 * n, method)

        diff = y2[::2] - y1
        err_runge = np.max(np.linalg.norm(diff, ord=np.inf, axis=1)) * (2 ** order) / (2 ** order - 1)

        p = np.log2(prev_err_r / err_runge)

        h = (b - a) / n
        rows.append([n, h, err_runge, p])
        prev_err_r = err_runge
    return pd.DataFrame(rows, columns=['n', 'h', 'error_runge', 'p_runge'])

In [7]:
for m in ['bdf4']:
    t0 = time()
    df = table_runge(f, y0, 0.0, 1.0, m, n_start=100_000, k_tables=6)
    t = time() - t0
    print(f"\nметод {m}, время: {t} с")
    print(df)


метод bdf4, время: 467.14749789237976 с
         n             h   error_runge   p_runge
0   100000  1.000000e-05  7.277201e-09       NaN
1   200000  5.000000e-06  9.842825e-09 -0.435689
2   400000  2.500000e-06  2.765895e-08 -1.490602
3   800000  1.250000e-06  4.194456e-08 -0.600738
4  1600000  6.250000e-07  9.157814e-08 -1.126520
5  3200000  3.125000e-07  2.496312e-07 -1.446723
